In [2]:
!pip install datasets transformers tokenizers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install -U tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import datasets 
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast
from transformers import AutoModelForTokenClassification 

In [5]:
hiner = datasets.load_dataset('cfilt/HiNER-original')

  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [7]:
def tokenize_and_modify_labels(samples):
  tokenized_sample = tokenizer(samples['tokens'], padding=True, truncation=True, is_split_into_words=True)
  labels = []

  for i,ner_tag in enumerate(samples['ner_tags']):
    sample_word_id = tokenized_sample.word_ids(i)
    label = [0]*len(sample_word_id)

    for j, x in enumerate(sample_word_id):
      try:
        if ner_tag[x] != 0:
          label[j] = ner_tag[x]
        else:
          label[j] == 23
      except:
        pass

    labels.append(label)

  tokenized_sample['labels'] = labels
  return tokenized_sample

In [31]:
tokenized_train = hiner['train'].map(tokenize_and_modify_labels, batched=True)
tokenized_valid = hiner['validation'].map(tokenize_and_modify_labels, batched=True)
tokenized_test = hiner['test'].map(tokenize_and_modify_labels, batched=True)

Map:   0%|          | 0/75827 [00:00<?, ? examples/s]

Map:   0%|          | 0/10851 [00:00<?, ? examples/s]

Map:   0%|          | 0/21657 [00:00<?, ? examples/s]

In [37]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=23)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
labels_dict = {
        "B-GAME":1,
        "B-LANGUAGE":2,
        "B-LITERATURE":3,
        "B-LOCATION":4,
        "B-MISC":5,
        "B-NUMEX":6,
        "B-ORGANIZATION":7,
        "B-PERSON":8,
        "B-RELIGION":9,
        "B-TIMEX":10,
        "I-FESTIVAL":11,
        "I-GAME":12,
        "I-LANGUAGE":13,
        "I-LITERATURE":14,
        "I-LOCATION":15,
        "I-MISC":16,
        "I-NUMEX":17,
        "I-ORGANIZATION":18,
        "I-PERSON":19,
        "I-RELIGION":20,
        "I-TIMEX":21,
        "O":22,
        "B-FESTIVAL":23,
    }